# Run a basic Hermes Hartree-Fock energy calculation on ligand conformers from Auto3d

In this notebook, we'll perform a hermes energy calculation on a ligand derived from a smiles string

In [ ]:
import json
import os
import sys
import tarfile

from pdbtools import *
import requests
from datetime import datetime
from pathlib import Path
import py3Dmol

import rush

## 0) Setup

In [ ]:
EXPERIMENT = "tengu-py-auto3d_hermes"
LIGAND_SMILES = "CC(=O)OC1=CC=CC=C1C(=O)O "
LIGAND = "ASPRIN"
TAGS = ["qdx", EXPERIMENT, LIGAND]

In [ ]:
# |hide
WORK_DIR = Path.home() / "qdx" / EXPERIMENT

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=True)

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(WORK_DIR / ".rush", exist_ok=True)
import sys

os.chdir(WORK_DIR)

In [ ]:
# Get our client, for calling modules and using the rush API
client = await rush.build_provider_with_functions(batch_tags=TAGS)

In [ ]:
# |hide
client.workspace = WORK_DIR
client.config_dir = WORK_DIR / ".rush"

## 1) Preparation
We want to convert our raw files into prepared qdxfs, with correct charges and missing residues filled

In [ ]:
help(client.auto3d)

Help on function auto3d in module rush.provider:

async auto3d(*args: [<class 'pathlib.Path'>, <class 'str'>, dict[str, ~T]], target: rush.graphql_client.enums.ModuleInstanceTarget | None = <ModuleInstanceTarget.NIX_SSH_3: 'NIX_SSH_3'>, resources: rush.graphql_client.input_types.ModuleInstanceResourcesInput | None = ModuleInstanceResourcesInput(gpus=1, gpu_mem=None, gpu_mem_units=None, cpus=None, nodes=None, mem=None, mem_units=None, storage=18, storage_units=<MemUnits.MB: 'MB'>, walltime=None, storage_mounts=None), tags: list[str] | None = None, restore: bool | None = None) -> [<class 'pathlib.Path'>, <class 'pathlib.Path'>]
    Generate 3D conformers from SMILES strings and other inputs
    
    Module version: `tengu-auto3d/f5f26308f74bde570c870ff0b5bb1be42ac9c712`
    
    QDX Type Description:
    
        molecule_file: @bytes;
        molecule_file_type: string;
        options: {
            capacity: u32?,
            convergence_threshold: f32?,
            enumerate_isomer: 

In [ ]:
ligand_path = client.workspace / "asprin.smiles"
with open(ligand_path, "w") as ligand_file:
    print(f"{LIGAND_SMILES} {LIGAND_SMILES}", file=ligand_file)

(conformer_sdf, conformer_qdxf) = await client.auto3d(
    ligand_path, "smi", {"k": 5, "use_gpu": True}
)

print(f"{datetime.now().time()} | Running ligand preparation!")

14:21:42.725295 | Running ligand preparation!


In [ ]:
try:
    await conformer_sdf.download(filename="01_prepared_ligand.sdf")
except FileExistsError:
    # we will raise an error if you try to overwrite an existing file; you can
    # force the file to overwrite by passing an absolute filepath instead
    pass

2024-01-29 14:22:13,088 - rush - INFO - Argument ba2f69f2-0fd3-407c-99e9-64c8287c2a7f is now ModuleInstanceStatus.RUNNING
2024-01-29 14:22:46,741 - rush - INFO - Argument ba2f69f2-0fd3-407c-99e9-64c8287c2a7f is now ModuleInstanceStatus.AWAITING_UPLOAD


You should visualize your prepared ligand to spot check any issues

In [ ]:
view = py3Dmol.view()
with open(client.workspace / "objects" / "01_prepared_ligand.sdf", "r") as f:
    view.addModel(f.read(), "sdf")
    view.setStyle({"stick": {}})
    view.zoomTo()
    view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## 2) Quantum energy calculation
Finally, we submit our fragmented protein for quantum energy calculation, with custom configuration.

In [ ]:
help(client.hermes_energy)

Help on function hermes_energy in module rush.provider:

async hermes_energy(*args: [<class 'pathlib.Path'>, typing.Optional[~T], typing.Optional[~T]], target: rush.graphql_client.enums.ModuleInstanceTarget | None = <ModuleInstanceTarget.NIX_SSH_2_GPU: 'NIX_SSH_2_GPU'>, resources: rush.graphql_client.input_types.ModuleInstanceResourcesInput | None = ModuleInstanceResourcesInput(gpus=4, gpu_mem=None, gpu_mem_units=None, cpus=None, nodes=None, mem=None, mem_units=None, storage=1034, storage_units=<MemUnits.MB: 'MB'>, walltime=None, storage_mounts=None), tags: list[str] | None = None, restore: bool | None = None) -> [<class 'pathlib.Path'>, <class 'pathlib.Path'>]
    Runs a HERMES energy calculation given a topology, and optionally model and keyword configurations.
    Will use the default model and keywords if none are provided
    
    Module version: `tengu-prelude/efc6d8b3a8cc342cd9866d037abb77dac40a4d56`
    
    QDX Type Description:
    
        input: @Conformer;
        model: {

In [ ]:
(conformer,) = await client.pick_conformer(conformer_qdxf, 0)
(hermes_energy, matrices) = await client.hermes_energy(
    conformer,
    {
        "basis": "STO-3G",
        "aux_basis": "6-31G",
        "method": "RIMP2",
    },  # configuration for a fast converging, low accuracy run
    {
        "guess": {},
        "scf": {
            "convergence_metric": "diis",
            "dynamic_screening_threshold_exp": 10,
            "ndiis": 8,
            "niter": 40,
            "scf_conv": 0.000001,
        },
        "frag": {
            "ngpus_per_node": 1,
            "fragmentation_level": 2,
            "dimer_cutoff": 5,
            "dimer_mp2_cutoff": 5,
            "method": "MBE",
            "fragmented_energy_type": "TotalEnergy",
        },
        "export": {},
        "debug": {},
    },
    restore=True,
)

In [ ]:
await hermes_energy.get()

2024-01-29 14:32:32,771 - rush - INFO - Argument 098a98c4-5a3c-4f25-8eb4-4f9a856424fc is now ModuleInstanceStatus.RESOLVING
2024-01-29 14:33:14,049 - rush - INFO - Argument 098a98c4-5a3c-4f25-8eb4-4f9a856424fc is now ModuleInstanceStatus.ADMITTED
2024-01-29 14:33:25,357 - rush - INFO - Argument 098a98c4-5a3c-4f25-8eb4-4f9a856424fc is now ModuleInstanceStatus.DISPATCHED
2024-01-29 14:33:32,050 - rush - INFO - Argument 098a98c4-5a3c-4f25-8eb4-4f9a856424fc is now ModuleInstanceStatus.RUNNING
2024-01-29 14:33:43,514 - rush - INFO - Argument 098a98c4-5a3c-4f25-8eb4-4f9a856424fc is now ModuleInstanceStatus.AWAITING_UPLOAD


{'energy': {'hf': 0.0, 'mp2_os': None, 'mp2_ss': None},
 'fragment_basis_functions': [{'n_occupied_basis_functions': 47,
   'n_virtual_basis_functions': 26,
   'total_n_basis_functions': 73}],
 'full_system_basis_functions': {'n_occupied_basis_functions': 47,
  'n_virtual_basis_functions': 26,
  'total_n_basis_functions': 73},
 'monomer_energies': {'hf': [0.0], 'mp2_os': [None], 'mp2_ss': [None]}}